# Install Packages

In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

In [2]:
!pip install --upgrade pyspark==3.2.0 --no-cache | tail -n 1

#If you are running this notebook in non IBM Watson Studio env then uncomment the below pip statements and run it
#!pip install --upgrade pandas==1.2.3 --no-cache | tail -n 1 
#!pip install --upgrade requests==2.23 --no-cache | tail -n 1
#!pip install numpy==1.20.1 --no-cache | tail -n 1
#!pip install SciPy --no-cache | tail -n 1
#!pip install lime --no-cache | tail -n 1

!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1b

## Action: restart the kernel!

In [1]:
import sklearn
import pandas as pd
import numpy as np
import urllib3, requests, json, os, ibm_watson_machine_learning

# Load Data

In [2]:
train = pd.read_csv('/project_data/data_asset/train_xgb.csv')
train = train.iloc[: , 1:]
train.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,6.7,2.5,5.8,1.8,virginica
1,5.1,3.7,1.5,0.4,setosa
2,5.0,3.2,1.2,0.2,setosa
3,7.2,3.0,5.8,1.6,virginica
4,5.1,3.8,1.5,0.3,setosa


# Configure Credentials

The following user input field requires Watson OpenScale credentials. Make sure the credentials you enter is able to **access Watson OpenScale**

If not, contact cluster admin to provide needed permissions

### WOS Credentials

In [3]:
# After adding each appropriate value, hit enter to add the next value
from IPython.display import clear_output

WOS_CREDENTIALS = {}
WOS_CREDENTIALS["username"] = input("Username : ")
WOS_CREDENTIALS["password"] = input("Password : ")
WOS_CREDENTIALS["url"] = os.environ['RUNTIME_ENV_APSX_URL']

#Clear output to hide sensitive information from being displayed
clear_output(wait=False)

### WML Credentials

In [4]:
WML_CREDENTIALS = {
                   "url": WOS_CREDENTIALS['url'],
                   "username": WOS_CREDENTIALS['username'],
                   "password": WOS_CREDENTIALS['password'],
                   "instance_id": "wml_local",
                   "version" : "4.5" #If your env is CP4D 4.x.x then specify "4.x.x" instead of "4.5"  
                  }

In [7]:
wml_client = ibm_watson_machine_learning.APIClient(WML_CREDENTIALS)

In [6]:
# Find and set the default space
space_name = "msk-test"
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["id"]
if space_id is None:
    space_id = wml_client.spaces.store(
        meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["id"]
wml_client.set.default_space(space_id)

'SUCCESS'

In [7]:
space_id

# R Software Spec

In [9]:
config_yaml = """
name: custom
channels:
  - conda-forge
dependencies:
  - r
  - libiconv
"""
with open("config.yaml", "w", encoding="utf-8") as f:
    f.write(config_yaml)

In [9]:
# create a custom software specification
# note, you can also create a custom software specificaton using the user interface from Watson Machine Learning.

software_spec_uid = wml_client.software_specifications.get_uid_by_name("custom-r")
if software_spec_uid != "Not Found":
    software_spec_details = wml_client.software_specifications.get_details(software_spec_uid)
    package_ext_uid = software_spec_details["entity"]["software_specification"]["package_extensions"][0]["metadata"]["asset_id"]
    wml_client.package_extensions.delete(package_ext_uid)
    wml_client.software_specifications.delete(software_spec_uid)

meta_props = {wml_client.package_extensions.ConfigurationMetaNames.NAME: "custom-r", wml_client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml"}
package_ext_details = wml_client.package_extensions.store(meta_props=meta_props, file_path="config.yaml")
package_ext_uid = wml_client.package_extensions.get_uid(package_ext_details)
meta_props = {
    wml_client.software_specifications.ConfigurationMetaNames.NAME: "custom-r",
    wml_client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": wml_client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")},
}
software_spec_details = wml_client.software_specifications.store(meta_props=meta_props)
software_spec_uid = wml_client.software_specifications.get_uid(software_spec_details)
wml_client.software_specifications.add_package_extension(software_spec_uid, package_ext_uid)

# Python Function

In [10]:
params = {
    "wml_credentials": WML_CREDENTIALS, 
    "space_id": space_id,
    "project_id": os.environ['PROJECT_ID'],
    "script": "predict-reg-1.R",
    "required_assets": ["iris_lr-1.model"],
    "r_packages": ["jsonlite"],
}

def r_function(params=params):
    """
    Deployable Python function which downloads the required assets from a WML deployment space and runs the specified R Script.
    """
    import os
    import json
    import subprocess
    from ibm_watson_machine_learning import APIClient

    wml_client = APIClient(params["wml_credentials"])
    wml_client.set.default_project(params["project_id"]) # project id to download the predict function
    script = params["script"]
    assets = {x["metadata"]["name"]: x["metadata"]["asset_id"] for x in wml_client.data_assets.get_details()["resources"]}

    # download required assets
    wml_client.data_assets.download(assets[script], script)
    for x in params["required_assets"]:
        wml_client.data_assets.download(assets[x], x)
    
    # install R and other libraries
    subprocess.run(["Rscript", "-e", 'install.packages(c("jsonlite", "data.table", "stringi", "stringr"), repos="https://cloud.r-project.org")'])
    if "xgboost" in params["r_packages"]:
        params["r_packages"].remove("xgboost")
        subprocess.run(["Rscript", "-e", 'install.packages("https://cloud.r-project.org/src/contrib/Archive/xgboost/xgboost_1.5.2.1.tar.gz", repos=NULL, type="source")'])
    subprocess.run(["Rscript", "-e", 'install.packages(c(' + ",".join([f'"{x}"' for x in params["r_packages"]]) + '), repos="https://cloud.r-project.org")'])

    def score(payload):
        inputs1 = payload["input_data"][0]["values"]
        inputs2 = json.dumps(inputs1)
        result = subprocess.run(["Rscript", "--vanilla", script, inputs2], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out = result.stdout.decode("utf-8")
        try:
            out = json.loads(out)
        except:
            out = result.stdout.decode("utf-8")
        err = result.stderr.decode("utf-8") 
        
        preds = []
        
        for x in out:
    
            preds.append([x["prediction"]])
    

        return {"predictions": [{"fields": ["prediction"], 
                                 "values": preds,
                                 "err": err}
                               ]}
        

    return score

# Python Function Deployment

In [11]:
meta_props = {
    wml_client.repository.FunctionMetaNames.NAME: "iris reg 0912",
    wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: wml_client.software_specifications.get_uid_by_name("custom-r")}
    
function_details = wml_client.repository.store_function(function=r_function, meta_props=meta_props) # Store the deployable function in your Watson Machine Learning repository

function_uid = wml_client.repository.get_function_id(function_details)
meta_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME: "iris reg 0912",
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {},
}
wml_client.deployments.create(function_uid, meta_props=meta_props)



#######################################################################################

Synchronous deployment creation for uid: 'efaf0bfa-d31d-4e70-aeb3-49093272b681' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
....................................................................................................................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='f15b3646-4984-43c9-a9e9-25461dca015e'
------------------------------------------------------------------------------------------------




{'entity': {'asset': {'id': 'efaf0bfa-d31d-4e70-aeb3-49093272b681'},
  'custom': {},
  'deployed_asset_type': 'function',
  'hardware_spec': {'id': 'f3ebac7d-0a75-410c-8b48-a931428cc4c5',
   'name': 'XS',
   'num_nodes': 1},
  'name': 'iris reg 0912',
  'online': {},
  'space_id': 'd4c5a38c-44e3-473f-a4ff-c43b3f9cd2d2',
  'status': {'online_url': {'url': 'https://internal-nginx-svc.zen.svc.cluster.local:12443/ml/v4/deployments/f15b3646-4984-43c9-a9e9-25461dca015e/predictions'},
   'serving_urls': ['https://internal-nginx-svc.zen.svc.cluster.local:12443/ml/v4/deployments/f15b3646-4984-43c9-a9e9-25461dca015e/predictions'],
   'state': 'ready'}},
 'metadata': {'created_at': '2022-09-12T23:30:02.783Z',
  'id': 'f15b3646-4984-43c9-a9e9-25461dca015e',
  'modified_at': '2022-09-12T23:30:02.783Z',
  'name': 'iris reg 0912',
  'owner': '1000331030',
  'space_id': 'd4c5a38c-44e3-473f-a4ff-c43b3f9cd2d2'},
 'system': {'warnings': [{'id': 'Deprecated',
    'message': 'online_url is deprecated and w

# Check Predictions

In [15]:
endpoint = 'https://dse-cpd45-cluster2.cpolab.ibm.com/ml/v4/deployments/f15b3646-4984-43c9-a9e9-25461dca015e/predictions?version=2022-09-12'

payload = {"input_data": [{"fields": ["Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width", "Species"],
                           "values":  [[1,2,3,4, "setosa"],[3,4,5,6, "versicolor"]]}]}

headers = {"Content-Type": "application/json", "Accept": "application/json", "Authorization": wml_client._get_headers()["Authorization"]}

In [16]:
response = requests.post(url=endpoint, headers=headers,json=payload, verify=False).json()

In [17]:
response

{'predictions': [{'fields': ['prediction'],
   'values': [[3.1812], [3.4677]],
   'err': ''}]}

### Input

{"input_data": [{"fields": ["Sepal.Length",
    "Sepal.Width",
    "Petal.Length",
    "Petal.Width"],
   "values": [[1,2,3,4, "setosa"],[3,4,5,6, "versicolor"]]}]}}]}

### Result

{'predictions': [{'fields': ['prediction'],
   'values': [[3.1812], [3.4677]],
   'err': ''}]}

# Go to next notebook!